<a href="https://colab.research.google.com/github/tnotstar/machine-learning-zoomcamp/blob/master/cohorts/2025/08-deep-learning/submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Cell 0: Library imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

from torchvision import transforms, datasets
from torch.utils.data import DataLoader

import os

In [23]:
# Cell 1: Reproducibility seed
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Cell 2: Download and Unzip Data
!wget -q -NL https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip -q -o -d . data.zip

In [26]:
# Cell 3: Model Architecture
class HairClassifier(nn.Module):
    def __init__(self):
        super(HairClassifier, self).__init__()
        # Input shape: (3, 200, 200)

        # 1. Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.relu = nn.ReLU()

        # 2. Max Pooling
        self.pool = nn.MaxPool2d(kernel_size=2)

        # 3. Flatten
        # We need to calculate the flattened size.
        # Input (200, 200) -> Conv (3x3) -> (198, 198) -> Pool (2x2) -> (99, 99)
        # 32 filters * 99 * 99
        self.flatten_size = 32 * 99 * 99

        self.flatten = nn.Flatten()

        # 4. Linear Layer 1
        self.fc1 = nn.Linear(self.flatten_size, 64)

        # 5. Output Layer
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize Model
model = HairClassifier().to(device)

# Q1: Loss Function
criterion = nn.BCEWithLogitsLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [27]:
# Cell 4: Question 2 - Count Parameters
# We can use the manual counting method provided in the prompt
total_params = sum(p.numel() for p in model.parameters())
print(f"Question 2 - Total parameters: {total_params}")

Question 2 - Total parameters: 20073473


In [28]:
# Cell 5: Data Preparation (Phase 1)
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Assuming the zip created a 'data' folder with 'train' and 'test' subfolders
train_dir = './data/train'
test_dir = './data/test'

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
validation_dataset = datasets.ImageFolder(root=test_dir, transform=train_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=2)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False, num_workers=2)

In [29]:
# Cell 6: Training Loop (Phase 1)
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

print("Starting training...")
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Starting training...
Epoch 1/10, Loss: 0.6654, Acc: 0.6262, Val Loss: 0.6102, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5479, Acc: 0.7087, Val Loss: 0.6303, Val Acc: 0.6468
Epoch 3/10, Loss: 0.4858, Acc: 0.7638, Val Loss: 0.6991, Val Acc: 0.5970
Epoch 4/10, Loss: 0.4806, Acc: 0.7650, Val Loss: 0.6097, Val Acc: 0.7015
Epoch 5/10, Loss: 0.4403, Acc: 0.7863, Val Loss: 0.6215, Val Acc: 0.6517
Epoch 6/10, Loss: 0.3493, Acc: 0.8500, Val Loss: 0.6453, Val Acc: 0.6716
Epoch 7/10, Loss: 0.3005, Acc: 0.8775, Val Loss: 0.6953, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2746, Acc: 0.8938, Val Loss: 0.6370, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2119, Acc: 0.9225, Val Loss: 0.6828, Val Acc: 0.7114
Epoch 10/10, Loss: 0.1419, Acc: 0.9525, Val Loss: 0.8263, Val Acc: 0.6965


In [31]:
# Cell 7: Answer Q3 and Q4
import numpy as np

# Question 3: Median of training accuracy
median_train_acc = np.median(history['acc'])
print(f"Question 3 - Median Training Acc: {median_train_acc:.2f}")

# Question 4: Standard Deviation of training loss
std_train_loss = np.std(history['loss'])
print(f"Question 4 - Std Training Loss: {std_train_loss:.3f}")

Question 3 - Median Training Acc: 0.82
Question 4 - Std Training Loss: 0.154


In [32]:
# Cell 8: Data Augmentation Setup
train_aug_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Reload the training dataset with the new transforms
# Note: We do NOT re-initialize the model or optimizer
train_dataset_aug = datasets.ImageFolder(root=train_dir, transform=train_aug_transforms)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True, num_workers=2)

# Create a new history dict for the continuation
history_aug = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

print("Continuing training with augmentation...")
# Run for another 10 epochs
for epoch in range(10):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader_aug: # Use the augmented loader
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train
    history_aug['loss'].append(epoch_loss)
    history_aug['acc'].append(epoch_acc)

    # Validation (Loader remains the same as before, no augmentation on test set)
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history_aug['val_loss'].append(val_epoch_loss)
    history_aug['val_acc'].append(val_epoch_acc)

    print(f"Aug Epoch {epoch+1}/10, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Continuing training with augmentation...
Aug Epoch 1/10, Loss: 0.6685, Acc: 0.6525, Val Loss: 0.6630, Val Acc: 0.6816
Aug Epoch 2/10, Loss: 0.5434, Acc: 0.7125, Val Loss: 0.6875, Val Acc: 0.6269
Aug Epoch 3/10, Loss: 0.5330, Acc: 0.7188, Val Loss: 0.5545, Val Acc: 0.7015
Aug Epoch 4/10, Loss: 0.5334, Acc: 0.7212, Val Loss: 0.5752, Val Acc: 0.7164
Aug Epoch 5/10, Loss: 0.4792, Acc: 0.7638, Val Loss: 0.5288, Val Acc: 0.7313
Aug Epoch 6/10, Loss: 0.4904, Acc: 0.7562, Val Loss: 0.4977, Val Acc: 0.7463
Aug Epoch 7/10, Loss: 0.4871, Acc: 0.7512, Val Loss: 0.4742, Val Acc: 0.7612
Aug Epoch 8/10, Loss: 0.4882, Acc: 0.7562, Val Loss: 0.4657, Val Acc: 0.7612
Aug Epoch 9/10, Loss: 0.4412, Acc: 0.7913, Val Loss: 0.6119, Val Acc: 0.6517
Aug Epoch 10/10, Loss: 0.4558, Acc: 0.7788, Val Loss: 0.5128, Val Acc: 0.7512


In [33]:
# Cell 9: Answer Q5 and Q6

# Question 5: Mean of test (validation) loss for all epochs trained with augmentations
mean_val_loss_aug = np.mean(history_aug['val_loss'])
print(f"Question 5 - Mean Test Loss (Augmented): {mean_val_loss_aug:.3f}")

# Question 6: Average of test (validation) accuracy for the last 5 epochs (6 to 10)
# Python list indexing: the last 5 items are [-5:]
avg_val_acc_last_5 = np.mean(history_aug['val_acc'][-5:])
print(f"Question 6 - Avg Test Acc (Last 5 epochs): {avg_val_acc_last_5:.2f}")

Question 5 - Mean Test Loss (Augmented): 0.557
Question 6 - Avg Test Acc (Last 5 epochs): 0.73
